Library

In [1]:
# from pyvi import ViTokenizer, ViPosTagger
import numpy as np
import pandas as pd
# import gensim
import sklearn
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam
from configs import ROOT_DIR
import csv
from utils.transformer import tokenizer


Data Preparation

In [2]:
# datanewscontent=pd.read_excel(r'path_to_newscontent.xlsx','Sheet1')
# datacomment=pd.read_excel(r'path_to_comments.xlsx','Sheet1')

dataset_file_path = ROOT_DIR + '\datamining\data\shopee\data.csv'

with open(dataset_file_path, encoding="utf-8") as fp:
    reader = csv.reader(fp, delimiter=",", quotechar='"')
    # next(reader, None)  # skip the headers
    data_read = [row for row in reader]
    
datarow = [r[0] for r in data_read]
labelrow = [r[1] for r in data_read]

print(len(datarow))
print(len(labelrow))

15030
15030


Tokenizer

In [3]:
datacmt=[]
labelcmt=[]
i=0
for d in datarow:
    if labelrow[i] == "1" :
        labelcmt.append(0)
        e=tokenizer(str(d))
        datacmt.append(e)
    if labelrow[i] == "3" :
        labelcmt.append(1)
        e=tokenizer(str(d))
        datacmt.append(e)
    if labelrow[i] == "5" :
        labelcmt.append(2)
        e=tokenizer(str(d))
        datacmt.append(e)
    i+=1
print(len(labelcmt))
print(len(datacmt))



9048
9048


In [5]:
# datanews=[]
# for d in datanewscontent['all_lower']:
#     e=ViTokenizer.tokenize(str(d))
#     datanews.append(e)
# labelnews=datanewscontent['label']

In [4]:
def truncatedvectors(data,n_components=300):
  svd_ngram = TruncatedSVD(n_components=n_components, random_state=42)
  svd_ngram.fit(data)
  return svd_ngram.transform(data)

TF-IDF

In [5]:
def tfidf(data):
  tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(1, 2))
  tfidf_vect_ngram.fit(data)
  X_data_tfidf_ngram =  tfidf_vect_ngram.transform(data)
  return truncatedvectors(X_data_tfidf_ngram)

In [8]:
X_data_tfidf_cmt=tfidf(datacmt)


Word2vec

In [9]:
# from gensim.models import Word2Vec,KeyedVectors 
# import os
# word2vec_model_path ="path_to_wikivimodel.bin"
# model = KeyedVectors.load_word2vec_format(word2vec_model_path,binary=True, unicode_errors='ignore')
# vocab = model.wv.vocab
# wv = model.wv

ModuleNotFoundError: No module named 'gensim'

In [0]:
# def get_word2vec_data(X):
#     word2vec_data = []
#     for x in X:
#         sentence = []
#         for word in x.split(" "):
#             if word in vocab:
#               sentence=sentence+wv[word].ravel().tolist()
#         word2vec_data.append(sentence)

#     return word2vec_data
# def change_to_word2vec(data):
#   data2vec=get_word2vec_data(data)
#   lengthOfdata=[len(data2vec[i]) for i,n in enumerate(data2vec)]
#   for i,n in enumerate(data):
#     if(len(data2vec[i])<max(lengthOfdata)):
#       for j in range(1,(max(lengthOfdata)-len(data2vec[i]))+1):
#         data2vec[i].append(0)
#   return truncatedvectors(np.array(data2vec))

In [0]:
# X_data_w2v_cmt=change_to_word2vec(datacmt)
# X_data_w2v_news=change_to_word2vec(datanews)

Bag Of Words

In [6]:
def bow(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(dataall)
  datacmtbow = tokenizer.texts_to_sequences(data)
  datacmtbow= pad_sequences(datacmtbow, maxlen=300)
  return datacmtbow

In [7]:
X_data_bow_cmt=bow(datacmt)
# X_data_bow_news=bow(datanews)

NameError: name 'dataall' is not defined

SVM

In [10]:
#Change: [X_data_tfidf_cmt,  X_data_w2v_cmt, X_data_bow_cmt],labelcmt , [X_data_tfidf_news,X_data_bow_news,X_data_w2v_news], labelnews
X_train, X_val, y_train, y_val = train_test_split(X_data_tfidf_cmt, labelcmt, test_size=0.2, random_state=42)

In [16]:
#kernel:linear, rbf
#gamma: [1e-3,5e-3,1e-2,5e-2,1e-1,5e-1,1e]
#C: [1,10,100,1000]
#You can change this parameters to have the best answer
clf=svm.SVC(kernel='linear',C=1000, gamma=1)
clf.fit(X_train, y_train)
val_predictions = clf.predict(X_val)
print(classification_report(val_predictions, y_val))

              precision    recall  f1-score   support

           0       0.77      0.71      0.74       904
           1       0.52      0.61      0.56       508
           2       0.75      0.74      0.74       398

    accuracy                           0.69      1810
   macro avg       0.68      0.68      0.68      1810
weighted avg       0.70      0.69      0.69      1810

